<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Advanced Compilation Options with H-Series

* [H-Series Hardware Compilation](#H-Series-Hardware-Compilation)
  * [Comnp.np.piling to H-Series Hardware Native Gates](#Comnp.piling-to-H-Series-Hardware-Native-Gates)
  * [The General SU(4) Entangler Gate](#The-General-SU(4)-Entangler-Gate)
  * [Controlling H-Series Hardware Compiler Optimizations](#Controlling-H-Series-Hardware-Compiler-Optimizations)
    * [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
    * [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)
* [Examples](#Examples)
    * [1. Benchmarking Circuit in non-native gate set](#1.-Benchmarking-Circuit-in-non-native-gate-set)
    * [2. QFT Circuits in native gate set](#2.-QFT-Circuits-in-native-gate-set)
    * [3. Circuit using $SU(4)$](#Circuit-using-$SU(4)$)

## H-Series Hardware Compilation

Native gates are gates on a quantum computer that the hardware physically executes. Different quantum computers may have different gates that are physically executed on the hardware. Writing a gate in a quantum circuit doesn't guarantee its physical execution on the device. For instance, on H-Series quantum computers, a Hadamard gate written in the circuit is not the actual gate executed. When users submit circuits using a Hadamard gate, the gate is translated into a $U1q$ gate followed by a $Rz$ gate, which the ion trap device physically executes. See the *System Model H1 Product Data Sheet* on the [System Model H1](https://www.quantinuum.com/hardware/h1) page or the *System Model H2 Product Data Sheet* on the [System Model H2](https://www.quantinuum.com/hardware/h2) page for a listing of the H-Series hardware native gates. 

The H-Series hardware compiler handles the translation from circuits users submit to the native gates run on hardware. In the H-Series Quantum Charge-Coupled Device (QCCD) architecture, the hardware compilation includes the assignment of which physical qubit corresponds to which qubit in a circuit as well as how qubits will be transported around the device. Since transport, as well as gating, incurs a small amount of error with each operation, the H-Series compiler aims to minimize the number of gates that need to be executed. 

<div style="text-align: center;">
         <img src="figureshseries-compilation-stac6.png" width="400" />
</div>

### Compiling to H-Series Hardware Native Gates

On the Quantinuum H-Series devices, there are different native two-qubit gates available. The default native two-qubit gates include a fully entangling two-qubit gate, $ZZ()$, and an arbitrary angle ZZ gate, $Rzz(\theta)$. Another native gate available on the hardware is the General $SU(4)$ Entangler gate.

By default, the hardware compiler compiles to the $ZZ()$ or $Rzz(\theta)$ gate. Currently, only one native gate can be specified at a time. This ensures everything aligns in the global operations of the circuit. 

If users would like to use the General $SU(4)$ Entangler gate and not have the circuit rebased to $ZZ()$ or $Rzz(\theta)$ by the hardware compiler, they need to specify the $SU(4)$ gate using the `nativetq` option. The `nativetq` option is available to override the hardware stack's default two-qubit gate and use the supplied gate instead.

* `nativetq`: override the stack's default native two-qubit gate and use the supplied gate as the gate instead
  * `ZZ`: compile circuit to the $ZZ$ gate
  * `RZZ`: compile circuit to the $Rzz(\theta)$ gate, known as the `Optype.ZZPhase` gate within `pytket`.
  * `Rxxyyzz`: compile circuit to the $SU(4)$ gate, known as `Optype.TK2` within `pytket`.

### The General $SU(4)$ Entangler Gate

The General $SU(4)$ Entangler gate, or `Rxxyyzz`, is available in TKET as [`OpType.TK2`](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.TK2). This gate is a combination of `OpType.XXPhase`, `OpType.YYPhase` and `OpType.ZZPhase`, and requires three angles as input, $\alpha$, $\beta$ and $\gamma$. The definition of the gate is provided below:

$$\begin{equation} \textrm{TK2}(\alpha, \beta, \gamma) = e^{-\frac{1}{2} i \pi \alpha (\hat{X} \bigotimes \hat{X})} \quad e^{-\frac{1}{2} i \pi \beta (\hat{Y} \bigotimes \hat{Y})} \quad e^{-\frac{1}{2} \pi \gamma (\hat{Z} \bigotimes \hat{Z})} = e^{-\frac{1}{2} i \alpha (\hat{X} \bigotimes \hat{X}) + \beta (\hat{Y} \bigotimes \hat{Y}) + \gamma (\hat{Z} \bigotimes \hat{Z})} \end{equation} $$

This gate can be used as follows within TKET.

In [ ]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.circuit import Circuit
from sympy import Symbol

symbols = [Symbol("a"), Symbol("b"), Symbol("c")]
circuit = Circuit(2)
circuit.TK2(*symbols, *circuit.qubits)
render_circuit_jupyter(circuit)

This circuit can be converted to a QASM string using the [`circuit_to_qasm_str`](https://tket.quantinuum.com/api-docs/qasm.html#pytket.qasm.circuit_to_qasm_str) function and by specifying the Quantinuum header `hqslib1`.

In [ ]:
from pytket.qasm.qasm import circuit_to_qasm_str

print(circuit_to_qasm_str(circuit, header="hqslib1"))

### Controlling H-Series Hardware Compiler Optimizations

Users have the option of submitting circuits using whichever quantum gate set they desire. Users do not need to think about which physical gates will be executed or how physical qubits will move around the device since the hardware compiler manages this. In certain cases, however, users may want to know that the circuit they submit is going to be run on the device exactly as they write it. For example, when running benchmarking circuits users may want circuits to be executed exactly as specified in the circuit even if its not the most optimal in total number of two-qubit gates. 

Within the Quantinuum stack, the ability to control levels of TKET optimizations and control over what is executed on the hardware is provided between 4 different job submission parameters in the API.

There are two ways to think about using these options:
1. [Circuits written in any gate set](#Circuits-written-in-any-gate-set)
2. [Circuits written in the hardware's native gate set](#Circuits-written-in-the-hardware's-native-gate-set)

#### Circuits written in any gate set

Users are free to submit circuits written with any gate set, not just the native gate set of the hardware. In this case, the options for control over what optimizations are applied are given at the TKET level. TKET will rebase the circuit to the native gate set it believes is most optimal and the hardware compiler will handle further optimizations of gate combinations as it applies to transport and ion assignment. **We recommend this for the majority of use cases.**

* `tket-opt-level`: the `tket` optimization level to apply (default: `2`), with `tket` optimizations turned on, the hardware compiler will provide further gate combination logic as makes sense for ions and transport
  * `2`: powerful optimizations, can use approximate methods, compilation can be expensive
  * `1`: basic optimization, compiles quickly
  * `0`: rebase the circuit with `tket`
  * `null`: rebase the circuit without `tket`, using the hardware compiler only

#### Circuits written in the hardware's native gate set

For circuits that are written only using gates in the hardware's native gate set, various levels of control are provided for what optimizations will be performed in the stack.

Users may still choose to apply `tket` optimizations specified above to their circuit and control this with `tket-opt-level`, even if the circuit is written in the native gate set since further reductions in the number of quantum gates may be found, which will improve results. 

To use the native gates and circuit as the user submits it `tket-opt-level` must be set to `null` when submitting and the following options must be used.

* `no-opt`: turns off all `tket` optimizations *and* all hardware compiler gate combination logic. (default: `False`)
    * If more than 1 native gate is used in the circuit, the circuit will be rebased to 1 native gate, but no further gate combination logic will occur.
    * The job will fail if `no-opt` is set to `True` and the circuit contains non-native gates. 
* `noreduce`: turns off all `tket` optimizations, all hardware compiler gate combination logic, and requires exact 1:1 correspondence of two-qubit gates with gates on the system. This requires the circuit be submitted using one 1 of the native two-qubit gates on the system, otherwise an error will be returned. (default: `False`)

Note that `tket-opt-level` set to `null`, `no-opt` set to `True`, and `noreduce` set to `True` all disable `tket` optimizations, but `no-opt` and `noreduce` also turn off all hardware gate combination logic.

**Create plot of quantum circuit -> tket opt or no-opt or nativetq or noreduce**

## Examples

Now we illustrate the above options with a few examples. First we import the functions we need in `pytket`.   

In [14]:
import numpy as np
import pandas as pd

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

### Quantum Volume Circuits with and without the General $SU(4)$ Entangler

Quantum volume is a benchmarking test that was initially proposed by IBM [TODO: insert reference]. It is a test that aims to verify the quality as well as the quantity of qubits on the machine. The test does this by peforming rounds of single and two qubit gates between random pairs of qubits for as many rounds as qubits in the test. For example, for quantum volume of 2^N where N=4, 4 random rounds of gates are performed. If the quantum computer performing the test passes, it verifies that the quantum computer can perform quality computation with equal depth as width of the circuit. The advantage to using quantum volume is that it gives users the confidence that not only do they have the number of qubits to support running their circuit, but the two-qubit gate fidelity is there to support circuits of significant depth as well. Quantum algorithms need not only qubits, but the ability to run many gates consecutively as well. Quantinuum has steadily been increasing the quantum volume of H-Series machines. 

In [2]:
qv_circuit_wSU4 = Circuit(4, name="QV Example with SU(4)")

# Round 1
qv_circuit_wSU4.U3(3.2465066100946998*np.pi, 2.5284408131911524*np.pi, 4.1734440424380335*np.pi, 0)
qv_circuit_wSU4.U3(3.56126038967905*np.pi, 0.5417259593027202*np.pi, 0.5889430256705375*np.pi, 1)
qv_circuit_wSU4.U3(3.4457004625001506*np.pi, 2.238803660106308*np.pi, 2.666707582855283*np.pi, 2)
qv_circuit_wSU4.U3(3.6550299908952426*np.pi, -0.09542555037191236*np.pi, 2.2959051047685524*np.pi, 3)

qv_circuit_wSU4.TK2(0.3747103659393297*np.pi, 0.25045542681642363*np.pi, 0.16594061155215553*np.pi, 0, 2)
qv_circuit_wSU4.TK2(0.4049821668324701*np.pi, 0.3201934298378358*np.pi, 0.14725271101077092*np.pi, 1, 3)

# Round 2
qv_circuit_wSU4.U3(3.9176110504261077*np.pi, 2.9748059513568568*np.pi, 3.5892285844018543*np.pi, 0)
qv_circuit_wSU4.U3(3.057201228666834*np.pi, 2.3577755337975868*np.pi, 3.7508467917045114*np.pi, 1)
qv_circuit_wSU4.U3(3.5721279036823654*np.pi, 0.05730037730060722*np.pi, 3.2791170965521705*np.pi, 2)
qv_circuit_wSU4.U3(3.535724388746909*np.pi, 0.05595930815342476*np.pi, 3.1299299416986477*np.pi, 3)

qv_circuit_wSU4.TK2(0.43955814978837976*np.pi, 0.3294379018024353*np.pi, 0.22483582064207827*np.pi, 0, 3)
qv_circuit_wSU4.TK2(0.30761522377699807*np.pi, 0.24902805179116916*np.pi, 3.9421010293613925*np.pi, 1, 2)

# Round 3
qv_circuit_wSU4.U3(3.024862371165571*np.pi, 2.3296051463212555*np.pi, 4.47465538899325*np.pi, 0)
qv_circuit_wSU4.U3(3.3793899755344796*np.pi, 1.2958915291565876*np.pi, 0.9370112066521646*np.pi, 1)
qv_circuit_wSU4.U3(3.466056716208809*np.pi, 3.4526574409721653*np.pi, 1.2690606129862783*np.pi, 2)
qv_circuit_wSU4.U3(3.3138153252630116*np.pi, 2.996929687445126*np.pi, 3.2634131850838957*np.pi, 3)

qv_circuit_wSU4.TK2(0.48089722439645044*np.pi, 0.21393048476072266*np.pi, 3.92648010233256*np.pi, 0, 2)
qv_circuit_wSU4.TK2(0.33895792181776946*np.pi, 0.2518811716342433*np.pi, 0.13342119970597022*np.pi, 1, 3)

# Round 4
qv_circuit_wSU4.U3(3.787489505597836*np.pi, -0.2026128585756619*np.pi, 1.1655980797683991*np.pi, 0)
qv_circuit_wSU4.U3(1.0261490063596848*np.pi, 0.16193977105724777*np.pi, 1.6325026187234268*np.pi, 1)
qv_circuit_wSU4.U3(3.330882266512557*np.pi, 2.2076389369717675*np.pi, 0.752751605442116*np.pi, 2)
qv_circuit_wSU4.U3(2.5144296195023776*np.pi, 0.04059434060036249*np.pi, 4.306225020434145*np.pi, 3)

qv_circuit_wSU4.measure_all()

render_circuit_jupyter(qv_circuit_wSU4)

In [3]:
qv_circuit_no_SU4 = Circuit(4, name="QV Example without SU(4)")

qv_circuit_no_SU4.U3(3.2465066100946993*np.pi,1.5284408131911527*np.pi,0.1734440424380339*np.pi, 0)
qv_circuit_no_SU4.U3(3.438739610320952*np.pi,0.45827404069728117*np.pi,1.5889430256705381*np.pi, 1)
qv_circuit_no_SU4.U3(3.445700462500151*np.pi,1.2388036601063086*np.pi,0.6667075828552846*np.pi, 2)
qv_circuit_no_SU4.U3(3.344970009104757*np.pi,1.095425550371913*np.pi,1.2959051047685524*np.pi, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.3747103659393289*np.pi, 0, 2)
qv_circuit_no_SU4.ZZPhase(0.4049821668324666*np.pi, 1, 3)

qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.2504554268164241*np.pi, 0, 2)
qv_circuit_no_SU4.ZZPhase(0.32019342983783544*np.pi, 1, 3)

qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.16594061155215475*np.pi, 0, 2)
qv_circuit_no_SU4.ZZPhase(0.14725271101077048*np.pi, 1, 3)

qv_circuit_no_SU4.U3(3.082388949573886*np.pi,0.025194048643138567*np.pi,1.5892285844018454*np.pi, 0)
qv_circuit_no_SU4.U3(3.057201228666816*np.pi,1.6422244662024277*np.pi,0.24915320829550014*np.pi, 1)
qv_circuit_no_SU4.U3(3.42787209631763*np.pi,0.9426996226994098*np.pi,1.279117096552166*np.pi, 2)
qv_circuit_no_SU4.U3(3.5357243887469094*np.pi,1.9440406918465785*np.pi,0.8700700583013419*np.pi, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.4395581497883781*np.pi, 0, 3)
qv_circuit_no_SU4.ZZPhase(0.30761522377700024*np.pi, 1, 2)

qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.3294379018024374*np.pi, 0, 3)
qv_circuit_no_SU4.ZZPhase(0.2490280517911696*np.pi, 1, 2)

qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.22483582064207847*np.pi, 0, 3)
qv_circuit_no_SU4.ZZPhase(3.942101029361395*np.pi, 1, 2)

qv_circuit_no_SU4.U3(3.0248623711655656*np.pi,0.6703948536786994*np.pi,1.525344611006704*np.pi, 0)
qv_circuit_no_SU4.U3(3.37938997553448*np.pi,0.7041084708434175*np.pi,1.0629887933478317*np.pi, 1)
qv_circuit_no_SU4.U3(3.4660567162088007*np.pi,1.5473425590278413*np.pi,0.7309393870137169*np.pi, 2)
qv_circuit_no_SU4.U3(3.313815325263007*np.pi,1.0030703125548746*np.pi,0.7365868149161*np.pi, 3)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.48089722439645505*np.pi, 0, 2)
qv_circuit_no_SU4.ZZPhase(0.33895792181777307*np.pi, 1, 3)

qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,0.0*np.pi,1.0*np.pi, 3)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(3.5*np.pi,1.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(0.2139304847607227*np.pi, 0, 2)
qv_circuit_no_SU4.ZZPhase(0.25188117163424417*np.pi, 1, 3)

qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 0)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 1)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 2)
qv_circuit_no_SU4.U3(0.5*np.pi,1.5*np.pi,0.5*np.pi, 3)
qv_circuit_no_SU4.ZZPhase(3.9264801023325635*np.pi, 0, 2)
qv_circuit_no_SU4.ZZPhase(0.13342119970597022*np.pi, 1, 3)

qv_circuit_no_SU4.U3(3.2125104944021645*np.pi,0.7973871414243372*np.pi,0.8344019202316*np.pi, 0)
qv_circuit_no_SU4.U3(1.9738509936403101*np.pi,1.1619397710572517*np.pi,1.367497381276573*np.pi, 1)
qv_circuit_no_SU4.U3(3.6691177334874423*np.pi,1.20763893697177*np.pi,1.2472483945578832*np.pi, 2)
qv_circuit_no_SU4.U3(2.4855703804976246*np.pi,1.0405943406003615*np.pi,0.6937749795658569*np.pi, 3)

qv_circuit_no_SU4.measure_all()

render_circuit_jupyter(qv_circuit_no_SU4)

In [ ]:
from pytket.circuit import OpType

from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumBackendCompilationConfig

In [39]:
machine = "H1-1E"

# This didn't do anything since TK2 isn't in the "_GATE_SET"
# config = QuantinuumBackendCompilationConfig(target_2qb_gate=OpType.TK2)
# backend = QuantinuumBackend(device_name=machine, compilation_config=config)

backend = QuantinuumBackend(device_name=machine)
backend.login()

Enter your Quantinuum email:  megan.l.kohagen@quantinuum.com
Enter your Quantinuum password:  ········


In [ ]:
# compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=0)

In [ ]:
# Add TK2 as a two-qubite gate in the backend's gate set
backend._gate_set.add(OpType.TK2)

# Set the target 2-qubit compilation gate
backend.set_compilation_config_target_2qb_gate(target_2qb_gate=OpType.TK2)

In [46]:
backend.two_qubit_gate_set

{<OpType.TK2: 41>, <OpType.ZZMax: 70>, <OpType.ZZPhase: 73>}

In [47]:
backend.required_predicates

[NoSymbolsPredicate,
 GateSetPredicate:{ Reset RangePredicate Rz PhasedX MultiBit ZZMax ClassicalExpBox Barrier ZZPhase TK2 WASM SetBits Measure CopyBits ExplicitPredicate ExplicitModifier },
 MaxNQubitsPredicate(20),
 MaxNClRegPredicate(120)]

In [48]:
compiled_circuit_wSU4 = backend.get_compiled_circuit(qv_circuit_wSU4, optimisation_level=0)
render_circuit_jupyter(compiled_circuit_wSU4)

In [51]:
backend.required_predicates

[NoSymbolsPredicate,
 GateSetPredicate:{ Reset RangePredicate Rz PhasedX MultiBit ZZMax ClassicalExpBox Barrier ZZPhase TK2 WASM SetBits Measure CopyBits ExplicitPredicate ExplicitModifier },
 MaxNQubitsPredicate(20),
 MaxNClRegPredicate(120)]

In [50]:
n_shots = 100
# backend.cost(qv_circuit_wSU4, n_shots=n_shots, syntax_checker="H1-1SC")
backend.cost(compiled_circuit_wSU4, n_shots=n_shots, syntax_checker="H1-1SC")

CircuitNotValidError: Circuit with index 0 in submitted does not satisfy GateSetPredicate:{ Reset RangePredicate Rz PhasedX MultiBit ZZMax ClassicalExpBox Barrier ZZPhase WASM SetBits Measure CopyBits ExplicitPredicate ExplicitModifier } (try compiling with backend.get_compiled_circuits first).

In [ ]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)

In [ ]:
import json
with open("pytket_example_job_handle.json", "w") as file:
    json.dump(str(handle), file)

In [ ]:
from pytket.backends import ResultHandle

In [ ]:
with open("pytket_example_job_handle.json") as file:
    handle_str = json.load(file)

In [ ]:
handle = ResultHandle.from_str(handle_str)
result = backend.get_result(handle)

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>